"""
# 🎯 Phase 1: Initial Model Training
**Author:** Tesnim
**Status:** ✅ COMPLETED

⚠️ This notebook was executed on Google Colab.
Training is complete. This is documentation only.

**Results:**
- Validation Accuracy: ~95%
- Model: model_phase1_final.keras
- Training completed on: [date]
"""

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:

import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib  #save
import json #save
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [ ]:
csv_path = "/content/drive/MyDrive/ML_GEST_PR/freshClass_DS_preprocessed.csv"
df = pd.read_csv(csv_path)
df

In [ ]:

df['image_path'] = df['image_path'].apply(lambda x: x.replace('\\', '/'))
df['full_path'] = "/content/drive/MyDrive/ML_GEST_PR/" + df['image_path']

In [ ]:
df = df[df['full_path'].apply(os.path.exists)].reset_index(drop=True)

print(f" Images valides: {len(df)}")
print(f" Nombre de classes: {df['label'].nunique()}")
print(f" Classes: {df['label'].unique()}")

In [ ]:

X = df['full_path'].values
y = df['label'].values

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

train_df = pd.DataFrame({'full_path': X_train, 'label': y_train})
val_df = pd.DataFrame({'full_path': X_val, 'label': y_val})
test_df = pd.DataFrame({'full_path': X_test, 'label': y_test})

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

In [ ]:

def custom_preprocess(img):
    """Normalise les images de [0,255] vers [-1,1]"""
    return img / 127.5 - 1.0

IMG_SIZE = 128
BATCH_SIZE = 64

# Générateur d'entraînement avec augmentation
train_gen = ImageDataGenerator(
    preprocessing_function=custom_preprocess,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
).flow_from_dataframe(
    train_df,
    x_col='full_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Générateur de validation (sans augmentation)
val_gen = ImageDataGenerator(
    preprocessing_function=custom_preprocess
).flow_from_dataframe(
    val_df,
    x_col='full_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Générateur de test (sans augmentation)
test_gen = ImageDataGenerator(
    preprocessing_function=custom_preprocess
).flow_from_dataframe(
    test_df,
    x_col='full_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
print(f" Générateurs créés!")
print(f" Nombre de classes: {NUM_CLASSES}")
print(f" Classes: {list(train_gen.class_indices.keys())}")

In [ ]:

def custom_preprocess(img):
    """Normalise les images de [0,255] vers [-1,1]"""
    return img / 127.5 - 1.0

IMG_SIZE = 128
BATCH_SIZE = 64

# Générateur d'entraînement avec augmentation
train_gen = ImageDataGenerator(
    preprocessing_function=custom_preprocess,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
).flow_from_dataframe(
    train_df,
    x_col='full_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Générateur de validation (sans augmentation)
val_gen = ImageDataGenerator(
    preprocessing_function=custom_preprocess
).flow_from_dataframe(
    val_df,
    x_col='full_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Générateur de test (sans augmentation)
test_gen = ImageDataGenerator(
    preprocessing_function=custom_preprocess
).flow_from_dataframe(
    test_df,
    x_col='full_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
print(f" Générateurs créés!")
print(f" Nombre de classes: {NUM_CLASSES}")
print(f" Classes: {list(train_gen.class_indices.keys())}")

In [ ]:
callbacks_phase1 = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/ML_GEST_PR/best_model_phase1.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history_phase1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks_phase1,
    verbose=1
)

# Sauvegarde Phase 1
print("\n Sauvegarde Phase 1...")
model.save('/content/drive/MyDrive/ML_GEST_PR/model_phase1_final.keras')
with open('/content/drive/MyDrive/ML_GEST_PR/history_phase1.pkl', 'wb') as f:
    pickle.dump(history_phase1.history, f)
# Version joblib 
joblib.dump(history_phase1.history, '/content/drive/MyDrive/ML_GEST_PR/history_phase1.joblib')
with open('/content/drive/MyDrive/ML_GEST_PR/class_indices.json', 'w') as f:
    json.dump(train_gen.class_indices, f)

print(" Phase 1 terminée et sauvegardée!")
print(f" Meilleure val_accuracy Phase 1: {max(history_phase1.history['val_accuracy'])*100:.2f}%")

In [ ]:

print(" Chargement du modèle et historique Phase 1...\n")

import tensorflow as tf
import joblib
import json
import pickle
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Charger le modèle Phase 1
model = tf.keras.models.load_model('/content/drive/MyDrive/ML_GEST_PR/model_phase1_final.keras')
print(" Modèle Phase 1 chargé!")

# 2. Charger l'historique (avec joblib ou pickle)
try:
    history_phase1_dict = joblib.load('/content/drive/MyDrive/ML_GEST_PR/history_phase1.joblib')
    print("Historique chargé (joblib)")
except:
    with open('/content/drive/MyDrive/ML_GEST_PR/history_phase1.pkl', 'rb') as f:
        history_phase1_dict = pickle.load(f)
    print(" Historique chargé (pickle)")

# 3. Charger les class_indices
with open('/content/drive/MyDrive/ML_GEST_PR/class_indices.json', 'r') as f:
    class_indices = json.load(f)

NUM_CLASSES = len(class_indices)
class_names = list(class_indices.keys())

print(f" Nombre de classes: {NUM_CLASSES}")
print(f" Classes: {class_names}")
print(f" Meilleure val_accuracy Phase 1: {max(history_phase1_dict['val_accuracy'])*100:.2f}%")